In [2]:
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDRegressor

from sklearn.metrics import mean_squared_error
import numpy as np

In [7]:
def load_housing_data():
     
    return pd.read_csv(Path("housing.csv"))

def train_test_split(X, y, test_ratio = 0.2):
    total_size = len(X)
    print(total_size)

    test_size = int(total_size * test_ratio)
    train_size = total_size - test_size

    np.random.seed(42)
    rnd_indices = np.random.permutation(total_size)

    X_train = X.iloc[rnd_indices[:train_size]]
    y_train = y.iloc[rnd_indices[:train_size]]
    X_test = X.iloc[rnd_indices[train_size:]]
    y_test = y.iloc[rnd_indices[train_size:]]

    return X_train, X_test, y_train, y_test
    
def fill_na(X, strategy = 'median'):
    #X: ndarray array of shape (n_samples, n_features)
    #return ndarray of shape (n_samples, n_features) with missing values filled by the strategy

    imputer = SimpleImputer(strategy = strategy)
    imputer.fit(X)

    return imputer.transform(X)

def get_outlier_indices(X):
    #X: ndarray of shape (n_samples, n_features)
    #y: label of shape (n_samples, k = 1)
    #return the X and y with ourliers dropped
    
    isolation_forest = IsolationForest(random_state = 42)
    outlier_pred = isolation_forest.fit_predict(X)

    return outlier_pred

def standard_scaler(X):
    #scaling all columns in X such that for each column, we have mean = 0, std = 1
    
    std_scaler = StandardScaler()
    return std_scaler.fit_transform(X)

def ordinal_encoder(df_one_column):
    #df_one_column: a dataframe with one categorical column
    #return an array of numbers representing categories 
    
    ordinal_encoder = OrdinalEncoder()
    return ordinal_encoder.fit_transform(df_one_column)

def one_hot_encoder(df_one_column):
    #df_one_column: a dataframe with one categorical column
    #return a 2d array of 0 / 1
    
    cat_encoder = OneHotEncoder(sparse = False)
    return cat_encoder.fit_transform(df_one_column)


def train(X_train, y_train):
    
    # defining parameter range 
    m = len(X_train)
    
    param_grid = {'alpha': [0.1/m, 1/m, 10/m, 100/m],  
                  'eta0': [1, 0.1, 0.01, 0.001, 0.0001], 
                  'penalty':['l2'],
                  'random_state': [42],
                  'max_iter':[1000]}  

    print("Training ...")
    grid = GridSearchCV(SGDRegressor(), param_grid, refit = True, verbose = 3,n_jobs=-1, cv = 3) 

    # fitting the model for grid search 
    grid.fit(X_train, y_train) 

    # print best parameter after tuning 
    print("Training done")
    print(grid.best_params_)
    return grid

#    grid_predictions = grid.predict(X_test) 
    
    
    
    #sgd_reg = SGDRegressor() #(penalty="l2", alpha=0.1 / m, tol=None, max_iter=1000, eta0=0.01, random_state=42)

    #sgd_reg.fit(X, y.ravel()) # y.ravel() because fit() expects 1D targets
    
def main():

    #1 load data
    
    housing = load_housing_data() # housing is a dataframe
    
    housing_X = housing.drop("median_house_value", axis=1)
    housing_y = housing["median_house_value"].copy()    
    
    #2 split train, test sets
    
    housing_Xtrain, housing_Xtest, housing_ytrain, housing_ytest = train_test_split(housing_X, housing_y, test_ratio = 0.2)
    
    #3 prepare data for training
    
    #---handle missing values
    
    housing_Xtrain_num = housing_Xtrain.select_dtypes(include=[np.number])
    housing_Xtest_num = housing_Xtest.select_dtypes(include=[np.number])
    
    housing_Xtrain_num_columns = housing_Xtrain_num.columns
    
    housing_Xtrain_num = fill_na(housing_Xtrain_num)
    housing_Xtest_num = fill_na(housing_Xtest_num)
    
    #---drop outlier in training sets

    outlier_indices = get_outlier_indices(housing_Xtrain_num)
    housing_Xtrain = housing_Xtrain.iloc[outlier_indices == 1]
    housing_ytrain = housing_ytrain.iloc[outlier_indices == 1]


    housing_Xtrain_num = pd.DataFrame(housing_Xtrain_num, columns=housing_Xtrain_num_columns)
    housing_Xtest_num = pd.DataFrame(housing_Xtest_num, columns=housing_Xtrain_num_columns)

    housing_Xtrain_num = housing_Xtrain_num.iloc[outlier_indices == 1]

    
    
    #---Scale
    
    housing_Xtrain_num = standard_scaler(housing_Xtrain_num)
    housing_Xtest_num = standard_scaler(housing_Xtest_num)
    
    #---handle text column
    
    
    
    #4 train
    
    final_model = train(housing_Xtrain_num, housing_ytrain.values)
    
    #5 evaluate the final model
    
    final_predictions = final_model.predict(housing_Xtest_num)
    final_rmse = mean_squared_error(housing_ytest, final_predictions, squared=False)
    print("RMSE: ", final_rmse) # prints 41424.40026462184
    
    #plot train dev losses
    
    
    